In [1]:
import numpy as np
import os
import matplotlib.pyplot as plt
import h5py
from scipy import interpolate
from ccfj import GetStationPairs
from ccfj import GetStationPairs
import pandas as pd
import yaml
import geopandas as gp
from geopy.distance import great_circle
import openpyxl
from openpyxl import load_workbook
from concurrent.futures import ThreadPoolExecutor

import time

In [2]:
import sys
sys.path.append(r'../tools_F-J/')
from toollib_standard import maplib
from toollib_standard import mathlib
from toollib_standard import filelib
from toollib_standard import stacklib
from toollib_standard import plotlib

In [3]:
with open('a-project_repar.yml', 'r', encoding='utf-8') as f:
    proj = yaml.load(f.read(), Loader=yaml.FullLoader)
name_project = proj['name']
#name_project = 'project_repartrition/repartrition_01-03/'               # Harmon server
name_project

'project_repartition_v4.0/output_repar_v9.2_01-01/'

In [4]:
with open('0_config.yml', 'r', encoding='utf-8') as f:
    dir_config = yaml.load(f.read(), Loader=yaml.FullLoader)
dir_project_workspace = dir_config['dir_project_workspace']
dir_CC_workspace = dir_config['dir_CC_workspace']
print('dir_CC_workspace: ', dir_CC_workspace)
print('dir_project_workspace: ', dir_project_workspace)
dir_project = os.path.join(dir_project_workspace, name_project)
print('dir_project: ', dir_project)

dir_CC_workspace:  /shdisk/rem2/Harmon/F-J/San/
dir_project_workspace:  /shdisk/rem2/Harmon/F-J/San/
dir_project:  /shdisk/rem2/Harmon/F-J/San/project_repartition_v4.0/output_repar_v9.2_01-01/


In [5]:
filename = dir_project+'Basic_info.yml'
with open(filename, 'r', encoding='utf-8') as f:
    info_basic = yaml.load(f.read(), Loader=yaml.FullLoader)
filename_bi = dir_project+'Basic_info.npy'
info_basic_bi = np.load(filename_bi, allow_pickle='TRUE').item()      # setting dictionary

In [6]:
key_subworks = info_basic['key_subworks']
key_subworks

['1--08-03',
 '2--09-03',
 '3--10-03',
 '4--11-03',
 '5--12-03',
 '6--13-03',
 '7--14-03',
 '8--15-03',
 '9--16-03',
 '10--18-03',
 '11--19-03',
 '12--20-03',
 '13--21-03',
 '14--22-03',
 '15--23-03',
 '16--24-03',
 '17--25-03',
 '18--26-03',
 '19--27-03',
 '20--28-03',
 '21--29-03',
 '22--30-03',
 '23--31-03',
 '24--32-03',
 '25--33-03',
 '26--34-03',
 '27--35-03',
 '28--36-03',
 '29--37-03',
 '30--41-03',
 '31--42-03',
 '32--43-03',
 '33--46-03',
 '34--47-03',
 '35--48-03',
 '36--52-03',
 '37--53-03',
 '38--07-04',
 '39--08-04',
 '40--09-04',
 '41--10-04',
 '42--12-04',
 '43--13-04',
 '44--17-04',
 '45--20-04',
 '46--21-04',
 '47--22-04',
 '48--23-04',
 '49--24-04',
 '50--25-04',
 '51--27-04',
 '52--28-04',
 '53--29-04',
 '54--30-04',
 '55--31-04',
 '56--32-04',
 '57--33-04',
 '58--34-04',
 '59--35-04',
 '60--36-04',
 '61--41-04',
 '62--42-04',
 '63--47-04',
 '64--48-04',
 '65--49-04',
 '66--52-04',
 '67--53-04',
 '68--08-05',
 '69--12-05',
 '70--13-05',
 '71--14-05',
 '72--16-05',
 

In [7]:
if 'key_subworks_repick' in info_basic.keys():
    key_subworks_repick = info_basic['key_subworks_repick']
else:
    key_subworks_repick = []
key_subworks_repick

[]

In [8]:
dir_stack = dir_project+info_basic['dir_stack']
dir_CC = dir_CC_workspace+info_basic['name_CC']
dir_partition = dir_project + info_basic['dir_partition']
#dir_CC = dir_CC_workspace+'CC/CC_40_prewhiten/'

In [9]:
stalistname_all = info_basic['stalistname_all']
stainfo = pd.read_excel(stalistname_all)
nsta_all = len(stainfo.iloc[:,0])
StationPairs_all = GetStationPairs(nsta_all)
nPairs_all = int(len(StationPairs_all)/2)
stalist_all = stainfo['Station'].tolist()
lat_all = stainfo['latitude'].tolist() 
lon_all = stainfo['longitude'].tolist()

In [10]:
ncffile = h5py.File( dir_CC+ 'gather_all.h5','r')
ncfs = ncffile['ncfs'][:]
f = ncffile['f'][:]
count_all = ncffile['count'][:]
ncffile.close()

In [11]:
info_basic_bi['r_max'] = {}

In [12]:
def linear_stack(key_subwork):
    global key_subworks
    global dir_stack
    global info_basic
    global stainfo_all
    global dir_partition
    global key_subworks_repick

    print("Collecting ",key_subwork,' ...')
    outname = str(key_subwork)+'_gather_linear.h5'
    if key_subwork in key_subworks_repick:
        if os.path.exists(dir_stack+outname):
            os.remove(dir_stack+outname)
    if os.path.exists(dir_stack+str(key_subwork)+'_gather_linear.h5'):
        print("File exists.")
        return

    time0 = time.time()

    nf = info_basic['nf']
    
    filepath = dir_partition + str(key_subwork) + '.txt'
    stalist, lat, lon = np.loadtxt(filepath, dtype='str', unpack=True)
    nsta = len(stalist)
    StationPairs = GetStationPairs(nsta)
    nPairs = int(len(StationPairs)/2)

    ncfs_sum_linear = np.zeros((nPairs,nf),dtype=np.complex64)
    r = np.zeros(nPairs)
    count= np.zeros(nPairs)
    StationPairs = GetStationPairs(nsta)
    for i in range(nPairs):
        sta1 = StationPairs[2*i]
        sta2 = StationPairs[2*i+1]
        idx1 = np.min( [int(stalist_all.index(stalist[sta1])),int(stalist_all.index(stalist[sta2]))] )
        idx2 = np.max( [int(stalist_all.index(stalist[sta1])),int(stalist_all.index(stalist[sta2]))] )
        #idx1 = int(stalist_all.index(stalist[sta1]))
        #idx2 = int(stalist_all.index(stalist[sta2]))
        #idx1 = int(stainfo[stainfo[key_pd]==stalist[sta1]].index.values[0])
        #idx2 = int(stainfo[stainfo[key_pd]==stalist[sta2]].index.values[0])
        
        m = 0
        for j in range(nsta_all-idx1,nsta_all):
            m += j
        num = m +idx2 - idx1 -1
        
        ncfs_sum_linear[i,:] = np.nan_to_num(ncfs[num,:])
        count[i] = count_all[num]
        #r[i] = r0[num]
        #if count_all[num] > 0:
        #    ncfs_sum_linear[i,:] = ncfs[num,:]/count_all[num]
        #    count[i] = count_all[num]

        r[i] = great_circle((lat[sta1],lon[sta1]),(lat[sta2],lon[sta2])).km
        #names.append([stalist_all[StationPairs_all[2*num]],stalist_all[StationPairs_all[2*num+1]]])

    
    ncffile = h5py.File(dir_stack+outname,'w')
    ncffile.create_dataset('ncfs',data=ncfs_sum_linear)
    ncffile.create_dataset('r',data=r)
    ncffile.create_dataset('count',data=count)
    ncffile.create_dataset('f',data=f)
    ncffile.create_dataset('StationPairs',data=StationPairs)
    #print(ncffile.keys())
    ncffile.close()
    #np.savez(dir_stack+key_subwork+"_summed-linear.npz",ncfs= ncfs_sum_linear,r = r,stalist=stalist,StationPairs=StationPairs)
    print("Done in ",time.time()-time0," s.", str(key_subworks.index(key_subwork)+1),'/',len(key_subworks))

In [13]:
nThreads = 60

In [14]:
for key_subwork in key_subworks:
    linear_stack(key_subwork)

Done in  0.3939096927642822  s. 1 / 597
Done in  0.42382144927978516  s. 2 / 597
Done in  0.6195242404937744  s. 3 / 597
Done in  0.7071216106414795  s. 4 / 597
Done in  0.799583911895752  s. 5 / 597
Done in  0.6575794219970703  s. 6 / 597
Done in  0.680830717086792  s. 7 / 597
Done in  0.5884904861450195  s. 8 / 597
Done in  0.7077338695526123  s. 9 / 597
Done in  0.9106781482696533  s. 10 / 597
Done in  0.7933177947998047  s. 11 / 597
Done in  1.434324026107788  s. 12 / 597
Done in  0.5974681377410889  s. 13 / 597
Done in  0.42818450927734375  s. 14 / 597
Done in  0.17055344581604004  s. 15 / 597
Done in  0.2434537410736084  s. 16 / 597
Done in  0.2575685977935791  s. 17 / 597
Done in  0.8605771064758301  s. 18 / 597
Done in  0.4675624370574951  s. 19 / 597
Done in  0.5241670608520508  s. 20 / 597
Done in  0.7163882255554199  s. 21 / 597
Done in  0.766425371170044  s. 22 / 597
Done in  0.354292631149292  s. 23 / 597
Done in  0.6353342533111572  s. 24 / 597
Done in  0.3603823184967041

In [15]:
np.save(filename_bi,info_basic_bi)